<a href="https://colab.research.google.com/github/CIKing123/2024CourseWork/blob/main/ModelTest3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opendatasets

In [2]:
import opendatasets as opd

In [3]:
# Gathering datasets
opd.download("https://www.kaggle.com/datasets/yamaerenay/100-images-of-top-50-car-brands")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jhlkj
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/yamaerenay/100-images-of-top-50-car-brands


100%|██████████| 47.2M/47.2M [00:00<00:00, 88.2MB/s]


In [4]:
from PIL import Image
import torch.nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import RandomAffine, RandomRotation, Compose
from torchvision.models import mobilenet_v2

In [5]:
carBrands= "/content/100-images-of-top-50-car-brands/imgs_zip/imgs"

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tqdm.keras import TqdmCallback

# Load and preprocess data
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '/content/100-images-of-top-50-car-brands/imgs_zip/imgs',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/content/100-images-of-top-50-car-brands/imgs_zip/imgs',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation')

# Load a pretrained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(50, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='mse', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=5,callbacks=[TqdmCallback(verbose=1)])

# Save the model
model.save('car_logo_shape_classifier.h5')


Found 3700 images belonging to 50 classes.
Found 897 images belonging to 50 classes.


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/5
30/58 [==============>...............] - ETA: 22:01 - loss: 0.0199 - accuracy: 0.0304

In [16]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Example image path
img_path = '/content/100-images-of-top-50-car-brands/imgs_zip/imgs/Bugatti/Bugatti_001.jpg'
img_array = preprocess_image(img_path)

In [17]:
# Predict the class
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

1/1 [==============================] - 0s 221ms/step
Predicted class: Citroen


In [ ]:
# Map the predicted class index to the class label
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}
predicted_label = class_labels[predicted_class[0]]

print(f'Predicted class: {predicted_label}')

# Load the original image
original_img = cv2.imread(img_path)
original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

# Draw a bounding box around the identified object
# For demonstration, we'll draw a box around the center; adjust as needed
(height, width, _) = original_img.shape
start_point = (int(width*0.3), int(height*0.3))
end_point = (int(width*0.7), int(height*0.7))
color = (255, 0, 0)  # Red color in BGR
thickness = 2

# Draw the rectangle
boxed_img = cv2.rectangle(original_img_rgb, start_point, end_point, color, thickness)

# Display the image
plt.figure(figsize=(8, 8))
plt.imshow(boxed_img)
plt.title(f'Predicted class: {predicted_label}')
plt.axis('off')
plt.show()